# Implementation Draft Details

### Notes:
There's some choice in how interprocess communication can be implemented (e.g. remote calls vs. channels). Given the choice between multiple options, my strategy will be to choose the simplest one, and note my choice here.
- **Work Messages**: Remote Calls / Futures vs. Channels. 
  - I'm not sure of the impact on performance when using a single channel for multiple processes that are possibly on different machines, so I will keep this as close to message passing by using **remote calls**.
  - **On second thought**, I'm not sure if this is the best way to pass work between processes since the communication paradigm isn't message passing but remote calls. The "messages" aren't just discrete units of data, they are more like tasks to be executed. Need to think of a natural way to incorporate the task scheduler on the workers, or potentially use channels for each worker (inefficient? Again, unsure of the particulars in e.g. cluster environments).
  - If I want this to resemble *message passing*, then channels seem like the way to go. There are a number of different ways to implement this overall scheme, I think, (and the documentation is frustratingly dispersed) so I need to figure out which way would be best.



1. **Idea**: Each process (controller and workers) may consist of multiple *tasks*, but they will be based around *RemoteChannels*. Each worker will have a work channel, which several tasks will use to:
  1. Take work and pass to compute task (same process)
  2. Signal the controller if work channel is empty or full, and if computation is happening
  3. Take work (remote process) and add to local channel (same process)

The main coordination construct is these remote channels. Assuming that take! and put! are atomic (I think they are), this should work.

**Main Questions/Comments**:
- The worker tasks will be telling the controller (via remotecall, not channels) that they are nonidle, and this remote call should return an idle node if there is one. *Considerations surrounding the global / local references to the idle list, and how the remotecall task accesses those*.
- If there is an idle node, then (since each worker has references to the other workers' work RemoteChannels) take! from local channel and put! in remote channel. *Are there sync concerns here...?*. Also, a worker status is based solely on the empty/nonempty status of the Work Channel. *Is this the best way to do this? Or does it matter if we **just** pulled an item of work off the queue, even tho the queue is empty?*
- If there is no idle node, then since the computation is performed in a different (local) task, wait for that task to complete, remove another item from work channel, then reassess if idle. This means that nonidle signals won't be sent to the controller more than once per completed work item.
- Worker tasks place the results fo their work into the Results RemoteChannel (passed to all worker processes)
- Controller waits until all worker processes are "idle" in the idle status list. This means that at the beginning, the initial worker process must be marked as nonidle. I also need to *make sure* that there are no corner cases where all processes may accidentally be marked as idle even though there is still work left.

For other load balancing methods, it is just a question of how the LB step changes. If the controller is sharing its idle status list, then there will be local copies (that will need to be synced). If the workers choose randomly, then there isn't much communication. If ...

## Strategies

### Scheduler Based (SB)
- **Controller**: Acts as a scheduler. Needs to:
  - Keep a list of idle and active workers
  - Receive status updates from workers
  - Match idle workers with active workers
- **Worker**: Performs computation. Needs to:
  - Send messages to and from controller
  - Request work from other worker
  - Perform computation
  - Send work to a requesting worker

### State Machines

**Controller Process**: 
- Create RemoteChannels
- Create workers; pass RemoteChannel references to each worker
- Initialize idle status list (*all controller tasks have access*)
- Pass all initial work to first worker
- 
- Wait for completion (what is signal?)

**Worker Process**: 
- While Not Finished:
  - If Working:
    - Mark as nonidle on controller, wait
    - (`tid = remotecall_fetch(nonidle, controller_id, ...)`)
    - If Has Extra Work:
      - Send work to idle node
      - (`remotecall(send_work, tid, work, ...)`)
    - Else:
      - Send "no work avail" to node
      - (`remotecall(send_work, tid, nowork, ...)`)
    - Do Work
  - Else:
    - Block / wait...?
    

## Example From Docs

In [3]:
function pmap(f, lst)
    np = nprocs()
    n = length(lst)
    results = Vector{Any}(n)
    i = 1
    # function to produce the next work item from the queue
    # in this case it's just an index
    nextidx() = (idx=i; i+=1; idx)
    @sync begin
        for p = 1:np
            if p != myid() || np == 1
                # if not the controller process (or controller is worker)
                @async begin
                    # create a local task that feeds work to other processes
                    while true
                        # local tasks are scheduled cooperatively, not preemptively,
                        # so they can be coordinated via nextidx()
                        idx = nextidx()
                        if idx > n
                            break
                        end
                        # perform computation on process p
                        # Note that context switching (between tasks) only occurs
                        # at well-defined points, as in this call to remotecall_fetch
                        results[idx] = remotecall_fetch(f, p, lst[idx])
                    end
                end
            end
        end
    end
    results
end

pmap (generic function with 1 method)

## Tasks

- Switching tasks does not use any space
- Switch among tasks can occur in any order


## Components